In [ ]:
# From: https://github.com/jupyter/notebook/issues/1000#issuecomment-359875246

import json
import os
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers

def notebook_path():
    """
    Return the full path of the jupyter notebook.
    """
    if 'NOTEBOOK_PATH' in os.environ:
        return os.environ['NOTEBOOK_PATH']
    kernel_id = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [ ]:
import datetime
import subprocess
from IPython.core.display import HTML

git_origin = 'https://github.com/hackenbergstefan/securec_ws1920'
git_revision = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode()

def print_metadata():
    return HTML('''Last modified: {now}<br/>
Revision: <a href="{git_origin}/blob/{git_rev}/{notebook_path}">git~{git_rev_short}<a>'''.format(
        now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        git_origin=git_origin,
        git_rev=git_revision,
        notebook_path=notebook_path(),
        git_rev_short=git_revision[:7],
    ))